In [1]:
from ase.io import read,write
gulp_structure=read('gulp.xyz',format='xyz')
atoms=read('../optimized.vasp',format='vasp')
gulp_structure.set_cell(atoms.cell)

write('gulp.dat',gulp_structure,format='lammps-data')
write('gulp.vasp',gulp_structure,format='vasp',vasp5=True)


In [2]:
natom=512
nmodes=natom*3

import numpy as np


In [3]:
dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))

In [4]:
dyn[0]

array([ 0.58531107, -0.01759416, -0.04394639, ...,  0.        ,
        0.        ,  0.        ])

In [5]:
from ase.io import read

atoms=read('../optimized.vasp',format='vasp')


In [6]:
masses=atoms.get_masses()


In [7]:
dyn[0:3,0:3]*masses[0]

array([[16.4384614 , -0.49413198, -1.23423436],
       [-0.49413198, 16.75081575, -1.31319478],
       [-1.23423436, -1.31319478, 15.1847998 ]])

In [8]:
dyn[0:3,243:246]*masses[0]

array([[-2.82429304,  0.9926882 ,  3.50022091],
       [ 1.41156586, -2.41028222, -4.05070798],
       [ 2.87561023, -2.75822616, -7.0644217 ]])

In [9]:
dyn[3:6,3:6]*masses[0]

array([[18.50484224, -0.35696175, -1.29725823],
       [-0.35696175, 15.33447207, -1.50714923],
       [-1.29725823, -1.50714923, 14.6965109 ]])

In [10]:
#reading phonopy format force constant
FC_file='/home/emi/GULP/thermal_conductivity/aSi512_GULP/FORCE_CONSTANTS_2ND'
with open(FC_file,'r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            #fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            fc_all[i,j]=fc
            start=start+nlineblock

In [11]:
fc_all[0,0]

array([[16.43823915, -0.49410715, -1.23422592],
       [-0.49410715, 16.7505942 , -1.31317768],
       [-1.23422592, -1.31317768, 15.18455583]])

In [12]:
fc_all[1,1]

array([[18.5043507 , -0.35710017, -1.29724485],
       [-0.35710017, 15.33418511, -1.50717439],
       [-1.29724485, -1.50717439, 14.6962958 ]])

In [13]:
fc_all[0,81]

array([[-2.82426989,  0.99268656,  3.50015051],
       [ 1.41156841, -2.41028727, -4.05062407],
       [ 2.87561549, -2.75820939, -7.06437718]])

以下のはまだテストしてない

In [14]:

'''
evaluate velocity operator.
structure file: unitcell structure with vasp POSCAR format
Dyn: Flat format (low:0x,0y,0z....., column:0x,0y,0z....) natom*3xnatom*3 Dynamical matrix
(already scaled by mass, regular output of lammps dynamical_matrix) 
'''
import sys
sys.path.append('../../src')
def get_Vij_from_flat(structure_file,Dyn):
    atoms=read(structure_file,format='vasp')
    natom=len(atoms.positions)

    dist=np.zeros((natom,natom,3))

    from nearest import find_nearest_ortho
    dist=np.zeros((natom,natom,3))
    positions=atoms.positions
    cell=atoms.cell
    for i in range(natom):  
        for j in range(i):
            dist[i,j]=find_nearest_ortho(positions,cell,i,j)
            #invert
            dist[j,i]=-dist[i,j]
    
    
    Rx=np.repeat(dist[:,:,0],3,axis=1)
    Rx=np.repeat(Rx,3,axis=0)
    Ry=np.repeat(dist[:,:,1],3,axis=1)
    Ry=np.repeat(Ry,3,axis=0)
    Rz=np.repeat(dist[:,:,2],3,axis=1)
    Rz=np.repeat(Rz,3,axis=0)  

    '''
    triu=np.triu(np.ones((natom*3,natom*3)))
    tril=np.tril(np.ones((natom*3,natom*3)),-1)
    mask=triu+tril*-1
    '''
    #Hadamard product
    Vx=Rx*Dyn*-1
    Vy=Ry*Dyn*-1
    Vz=Rz*Dyn*-1

    return Vx, Vy, Vz

In [15]:
import sys
sys.path.append('../../src')

In [16]:
import numpy as np

from nearest import find_nearest_ortho
dist=np.zeros((natom,natom,3))
positions=atoms.positions
cell=atoms.cell
for i in range(natom):  
    for j in range(i):
        dist[i,j]=find_nearest_ortho(positions,cell,i,j)
        #invert
        dist[j,i]=-dist[i,j]

In [17]:
Rx=np.repeat(dist[:,:,0],3,axis=1)
Rx=np.repeat(Rx,3,axis=0)
Ry=np.repeat(dist[:,:,1],3,axis=1)
Ry=np.repeat(Ry,3,axis=0)
Rz=np.repeat(dist[:,:,2],3,axis=1)
Rz=np.repeat(Rz,3,axis=0)  

In [18]:
Rx.shape

(1536, 1536)

In [19]:
Rx[0,0:6]

array([  0.        ,   0.        ,   0.        , -10.14206505,
       -10.14206505, -10.14206505])

In [20]:
Rx[0,243:260]

array([-1.09760464, -1.09760464, -1.09760464, -2.21248582, -2.21248582,
       -2.21248582, -8.17101401, -8.17101401, -8.17101401,  4.01047091,
        4.01047091,  4.01047091, -5.2321277 , -5.2321277 , -5.2321277 ,
       -5.81435552, -5.81435552])

In [21]:
Rx[1,0:6]

array([  0.        ,   0.        ,   0.        , -10.14206505,
       -10.14206505, -10.14206505])

In [22]:
Rx[3,0:6]

array([10.14206505, 10.14206505, 10.14206505,  0.        ,  0.        ,
        0.        ])

In [23]:
GULP_distx=np.loadtxt('../../aSi512_GULP/distx.dat')[:,2]
GULP_disty=np.loadtxt('../../aSi512_GULP/disty.dat')[:,2]
GULP_distz=np.loadtxt('../../aSi512_GULP/distz.dat')[:,2]

In [24]:
GULP_distx.shape

(130816,)

In [25]:
distx=[]
for i in range(natom):
    for j in range(i):
        distx.append(dist[i,j,0])

In [26]:
len(distx)

130816

In [27]:
distx[0], GULP_distx[0]

(10.142065046999988, 10.1420650461696)

In [28]:
np.max(np.abs(np.array(distx)-GULP_distx))

9.969545189392193e-10

In [29]:
structure_file='../optimized.vasp'
Dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))
Vx,Vy,Vz=get_Vij_from_flat(structure_file,Dyn)


In [30]:
Vx[0,243:246]

array([-0.11037768,  0.03879577,  0.13679397])

In [31]:
dist[81,0,0]

1.0976046420000003

In [32]:
dyn[0,243:260]*masses[0]

array([-2.82429304,  0.9926882 ,  3.50022091,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [33]:
dyn[0,243:260]*dist[0,81,0]*-1

array([-0.11037768,  0.03879577,  0.13679397,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [34]:
testVx=Dyn*Rx*-1

In [35]:
testVx[0,243:246]

array([-0.11037768,  0.03879577,  0.13679397])

In [36]:
#check

#check
GULP_Vijx=np.loadtxt('../../aSi512_GULP/Vijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vx-GULP_Vijx) > 0.1)

(array([318]), array([117]))

In [37]:
GULP_Vijx[318,117]

0.127553727030799

In [38]:
Vx[318,117]

-0.0

In [41]:
dyn[318,117]*masses[0]

0.0

In [44]:
fc_flat=np.reshape(fc_all.transpose(0,2,1,3),(natom*3,natom*3))

In [45]:
fc_flat[318,117]

-0.709636853283

In [50]:
index=np.where(np.abs(fc_flat-dyn*masses[0])>0.1)

In [51]:
fc_flat[index]

array([-0.18751268,  0.12030477, -0.10443746,  0.1508732 , -0.12547375,
       -0.2290738 ,  0.12727899,  0.11403424, -0.1329453 , -0.17011364,
       -0.20477279,  0.15424688, -0.70963685, -0.17575628,  0.11371868,
        0.13584233, -0.34352625, -0.2192917 , -0.26498115, -0.12278619,
        0.11414313, -0.2643091 ,  0.1039606 , -0.11056896,  0.18035842,
       -0.13599471,  0.10398358,  0.11448346,  0.26430877,  0.46748373,
       -0.23355415,  0.10335123])

In [53]:
dyn[index]*masses[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

ごく一部でForce constantが合致していないようだ

In [39]:
GULP_Vijy=np.loadtxt('../../aSi512_GULP/Vijy.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vy-GULP_Vijy) > 0.1)

(array([], dtype=int64), array([], dtype=int64))

In [40]:
GULP_Vijz=np.loadtxt('../../aSi512_GULP/Vijz.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vz-GULP_Vijz) > 0.1)

(array([], dtype=int64), array([], dtype=int64))